## Setup

Change the following variable settings to match your deployed model's *Inference endpoint*. for example: 

```
deployed_model_name = "fraud"
infer_endpoint = "https://fraud-predictor-userx-workshop.apps.clusterx.sandboxx.opentlc.com"
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import numpy as np
import requests

In [ ]:
deployed_model_name = "music"
infer_endpoint = "https://music-predictor-<project-name>.<cluster>.com"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

## Request Function

Build and submit the REST request. 

Note: You submit the data in the same format that you used for an ONNX inference.

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "dense_input",
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [ ]:
#Load the scaler and label encoder
with open('artifact/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)
    
with open('artifact/label_encoder.pkl', 'rb') as handle:
    label_encoder = pickle.load(handle)

In [ ]:
song_properties = pd.read_parquet('../1-data_prep/song_properties.parquet')
favorite_song = song_properties.loc[song_properties["name"]=="Not Like Us"]
favorite_song

In [ ]:
data = favorite_song[['is_explicit', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
prediction = rest_request(scaler.transform(data.values).tolist()[0])

In [ ]:
plt.figure(figsize = (6, 8))
plt.bar(x = range(len(prediction)),
        height = prediction)
plt.title('Prediction over countries', size = 12, weight = 'bold')
plt.show()

In [ ]:
most_likely_country = np.argmax(prediction)
country_code = label_encoder.inverse_transform([most_likely_country])
print(f"The most likely country is #{most_likely_country} which corresponds to the country code '{country_code[0]}'")